In [ ]:
 #In the following piece of code,i define the functions that it will use in the MMC algorithim 
 #Run it the following piece of code for obtein a protein configuration with a plot  and the sequence store

In [1]:
#Complete code
import numpy as np
from copy import deepcopy
from vpython import *
from numpy import *
from random import randint
import random

"""
plotfinalconformatoion:This function takes as an input a protein information and the grid information and it gives back
a plot.
as I had problems with the vpython and the cliking graph I had no choice but to dirty the code creating the net function within the function itself, although it is also defined a few lines below
This function plots the protein configuration with the energy for whichever specific protein.
"""


def plotfinalconformation(DDplot):
    M=[] 
    graph1=canvas(width=Maxx,height=Maxy,title='Self Avoiding Random Walk',range=ran)      
    positions=points(color=color.cyan,radius=2)
   
         
                       # to plot red or white points
    def net():                          #plots points in the visualization  
        for j in range(0,size):  
            yp=-4*j+size2                  # y world to screen coord. transformation
            for i in range (0,size):    #horizontal row
                xp=4*i-size2               # x world to screen coord. transformation
            
                positions.append(vector(xp,yp,0))
    net()
    length=len(DDplot)-1
    for count in range(0,length+1):    #loop for knowing the xcord,ycord and color 
        i=DDplot[count][0]
        j=DDplot[count][1]
        r=DDplot[count][2]
        if  r==2:
            col=vector(1,0,0)       # hydrophobic color red           
        else:
            col=vector(1,1,1)       # polar color white
            
        xp=4*i-size2             # coordinate transformation (plotting purposes)
        yp=-4*j+size2                   
        if count>=1:
            curve(pos=[vector(xol,yol,0),vector(xp,yp,0)]) #plot link last position to new one (plotting purposes)

        M=M +[points(pos=vector(xp,yp,0), color=col,radius=5)] #plot the current point (plotting purposes)
        xol=xp                          #start of new line
        yol=yp   #y calculated is now old y (for connecting the points in the next loop iteration)
        

"""
Selectcol(): compares a random number to a weighting factor, and based on the comparison (i.e. larger or smaller) returns 
a color vector (white or red) and a numerical value (1 or 2) for P and H monomers, resp. 
"""

def selectcol():         
   #Q1: Build in your random choice of H/P monomer in the below function, together with a weighting factor
    probabilityH=0.7        #insert a number between 0 and 1 fr the probability of the H monomer
                             #probabilityP=1-probabilityH

    choice=random.random()   

    if  choice<probabilityH:
        col=vector(1,0,0)       # hydrophobic color red
        r=2                     # for hydrophobic
    else:
        col=vector(1,1,1)       # polar color white
        r=1                     # for polar
    return col,r

"""
findrest(indexNumberofMonomer,totalProteinLength,xCoordinate,yCoordinate): initializes an energy value, and then loops
over the length of the protein. In the loop, it checks whether the coordinates  xCoordinate and yCoordinate that you 
inserted match those of any of the monomers. If that is the case (and the monomer == H), then a bond is formed, and
energy is set to 1, and this value is returned by the function

"""
def findrest(m,lengthfindrest,fin,fjn,proteininfofindrest):  

    ener=0                       # Initialize energy
    for t in range(m,lengthfindrest+1):  # next link not considered
        #given input coordinates (fin), (fjn) check all following monomers for whether they are at this position, and are H
        if proteininfofindrest[t][0]==fin and proteininfofindrest[t][1]==fjn and proteininfofindrest[t][2]==2: 
            ener=1               # if a match is found; return an energy of 1
   
    return ener
        
    
"""
findenergy(totalproteinLength,arrayContainingAllMonomerCoordinatesAndTheirColor): this function does the following:
-Loops over all monomers in the protein chain
-For each monomer, stores the x-coordinate and y-coordinate in i and j, and the color in cl,
-Checks whether the monomer is polar (P), if so: do nothing
-Checks whether the monomer is apolar (H), if so: do:
    -Check all neighbors of the H-monomer: 
        -First, hop one site to the left and check for existence of an H monomer using findrest()
              -Add outcome of findrest(), either 0 or 1 to the total energy of the chain
              -if it is an H-monomer, Use a little coordinate shift and plot a yellow dot halfway between the two H-beads
        -Second, hop one site to the right, check for existence of an H- monomer using findrest(), perform same steps
        -Third, idem but for one site upwards
        -Fourth, idem but for one site downwards
        
Once all is checked, the function returns the total chain energy
             
"""
def findenergy(lengthprotein,proteininfo):       # finds energy of each link
    energy=0                     # begins with no energy    
    size2=size*2
    for n in range (0,lengthprotein+1):
        i=proteininfo[n][0]
        j=proteininfo[n][1]
        cl=proteininfo[n][2]       
        if cl==1:    # if white (i.e. P), no energy is added to the total
            pass
        else:        # red 
            if n<lengthprotein+1:
                imin=int(i-1)        # check neighbour i-1,j
                js=int(j)
                if imin>=0:
                    e=findrest(n+2,lengthprotein,imin,js,proteininfo) # return energy 1 
                    energy=energy-e
                    if e==1:    # plot yellow dot in midle of neighbourhola
                            xol=4*(i-0.5)-size2             #always start at middle, x coord. transform 4*i-size2
                            yol=-4*j+size2                    # y coord transform -4*j+size2
                       
                            points(pos=vector(xol,yol,0),color=color.yellow, radius=4) #this part plots a yellow dot to indicate hydrogen bonding.
              

                ima=int(i+1)
                js=int(j)
                if ima<=size-1:      #  # check neighbour i+1,j
           
                    e=findrest(n+2,lengthprotein,ima,js,proteininfo)
                    energy=energy-e
                    if e==1:   
                            xol=4*(i+0.5)-size2             
                            yol=-4*j+size2 
                            
                            points(pos=vector(xol,yol,0),color=color.yellow, radius=4)
                iss=int(i)
                jma=int(j+1)
                if jma<=size-1:   #check neighbour i,j+1
                    e=findrest(n+2,lengthprotein,iss,jma,proteininfo)
                    energy=energy-e       
                    if e==1:      
                            xol=4*i-size2             #always start at middle, x coord. transform 4*i-size2
                            yol=-4*(j+0.5)+size2                    # y coord transform -4*j+size2
                       
                            points(pos=vector(xol,yol,0),color=color.yellow, radius=4)
                    
                    
                iss=i
                jmi=j-1
                if jmi>=0:        # check neighbor i, j-1
           
                     e=findrest(n+2,lengthprotein,iss,jmi,proteininfo)
                     energy=energy-e 
                     if e==1:    # plot yellow dot in midle of neighbour
                            xol=4*i-size2             #always start at middle, x coord. transform 4*i-size2
                            yol=-4*(j-0.5)+size2          
                            points(pos=vector(xol,yol,0),color=color.yellow, radius=4)
   
    return energy   # after checking all neighbours of each element in chain, the function returns the energy

"""
findenergy2: this function does the same as findenergy but wihout plotting a yellow point into the graphic
i create it for optimiaztion purpose when i run the code
"""        
            

def findenergy2(lengthprotein,proteininfo):       # finds energy of each link
    energy=0                     # begins with no energy     
    
    for n in range (0,lengthprotein+1):
        i=proteininfo[n][0]
        j=proteininfo[n][1]
        cl=proteininfo[n][2]       
        if cl==1:    # if white (i.e. P), no energy is added to the total
            pass
        else:        # red 
            if n<lengthprotein+1:
                imin=int(i-1)        # check neighbour i-1,j
                js=int(j)
                if imin>=0:
                    e=findrest(n+2,lengthprotein,imin,js,proteininfo) # return energy 1 
                    energy=energy-e
                   
              

                ima=int(i+1)
                js=int(j)
                if ima<=size-1:      
                    e=findrest(n+2,lengthprotein,ima,js,proteininfo)
                    energy=energy-e
                    
                iss=int(i)
                jma=int(j+1)
                if jma<=size-1:   #check neighbour i,j+1
                    e=findrest(n+2,lengthprotein,iss,jma,proteininfo)
                    energy=energy-e       
                    
                    
                    
                iss=i
                jmi=j-1
                if jmi>=0:        # check neighbor i, j-1
            #Q2: Finish this conditional statement for checking upper neighbor (i,j-1), such that it 
            #    adds the correct amount of energy if this is the case.
                     e=findrest(n+2,lengthprotein,iss,jmi,proteininfo)
                     energy=energy-e 
                     
 
    return energy   # after checking all neighbours of each element in chain, the function returns the energy

"""
function net()mit is used for plotting the grid,we will turn of for improve the optimizacion of the program
""" 

def net():                          #plots points in the visualization  
    for j in range(0,size):  
        yp=-4*j+size2                  # y world to screen coord. transformation
        for i in range (0,size):    #horizontal row
            xp=4*i-size2               # x world to screen coord. transformation
            
            positions.append(vector(xp,yp,0))



"""
kinkjump montecarlo move
"""
def kinkjump(DDk,gridk):  
    newDDk=[]
    size=len(gridk)                        #we need to know the length of the grid for the special conditions on the edge
    newgridk=zeros((size,size))  
    pickmon=randint(0,(len(DDk)-1))    #we generate a randon number between 0 and the chain length(both include).This will be helpful for pick later a random monomer of the chain
    xcord=DDk[pickmon][0]
    ycord=DDk[pickmon][1]
    newcord_x=0
    newcord_y=0
    if pickmon==0:           #we generate especial variables if we are working with the first monomer
        
        xcordnb=DDk[pickmon+1][0]
        ycordnb=DDk[pickmon+1][1]
    
    if pickmon==(len(DDk)-1):        #we generate especial variables if we are working with the last monomer
        xcordnb=DDk[pickmon-1][0]
        ycordnb=DDk[pickmon-1][1]
        
    if (xcord-1>=0 and ycord-1>=0 and xcord+1<size and ycord+1<size):         #First of all we need to check we are always working inside the grid(not outside the edge)
        if (gridk[xcord-1,ycord+1]==0 or gridk[xcord+1,ycord+1]==0 or gridk[xcord-1,ycord-1]==0 or gridk[xcord+1,ycord-1]==0):    #chek if there is some free grid point where the kink jump  can take place

                if (pickmon==0 or pickmon==(len(DD)-1)):  #if pickmon is the first or final monomer,we will try to apply the end rotation   
                    if (xcordnb==xcord and ycordnb==ycord-1):    #using the special variable we can know which is the coordinate of the neigboor monomer(i.e the coordinate of the second or the penultimate monomer)
                            if (gridk[xcordnb-1,ycordnb]==0 and gridk[xcordnb+1,ycordnb]==0): #if there is empty space where the end rotation would have take place (we will know which grid point to check  using the information of the neighboor monomer and choosing the ex)
                                if randint(0,1)==0:  #the two only posiible moves are availabe,choose one randomly
                                #move to diagonally, down and left
                                    newcord_x=xcord-1         #we obtain the new coordinate of the mononer for end-rotation
                                    newcord_y=ycord-1

                                else:
                                #move to diagonally, down and right
                                    newcord_x=xcord+1
                                    newcord_y=ycord-1

                            if (gridk[xcordnb-1,ycordnb]==0 and gridk[xcordnb+1,ycordnb]!=0):   #only one of the two posibility
                                #move to diagonally, down and left
                                    newcord_x=xcord-1                                   #we choose that posibility
                                    newcord_y=ycord-1
                            if (gridk[xcordnb+1,ycordnb]==0 and gridk[xcordnb-1,ycordnb]!=0):    #the same but with the other posibilty
                                    newcord_x=xcord+1
                                    newcord_y=ycord-1 
                                #move to diagonally, down and right

                    if (xcordnb==xcord and ycordnb==ycord+1):                #same as upward
                            if (gridk[xcordnb-1,ycordnb]==0 and gridk[xcordnb+1,ycordnb]==0):
                                if randint(0,1)==0:
                                #move to diagonally, up and left
                                    newcord_x=xcord-1
                                    newcord_y=ycord+1
                                else:
                                    newcord_x=xcord+1
                                    newcord_y=ycord+1
                                #.....etc
                            if (gridk[xcordnb-1,ycordnb]==0 and gridk[xcordnb+1,ycordnb]!=0):
                                    newcord_x=xcord-1
                                    newcord_y=ycord+1
                            if (gridk[xcordnb+1,ycordnb]==0 and gridk[xcordnb-1,ycordnb]!=0):
                                    newcord_x=xcord+1
                                    newcord_y=ycord+1

                    if (xcordnb==xcord+1 and ycordnb==ycord):
                            if (gridk[xcordnb,ycordnb+1]==0 and gridk[xcordnb,ycordnb-1]==0):
                                if randint(0,1)==0:
                                    newcord_x=xcord+1
                                    newcord_y=ycord+1
                                else:
                                    newcord_x=xcord+1
                                    newcord_y=ycord-1

                            if (gridk[xcordnb,ycordnb+1]==0 and gridk[xcordnb,ycordnb-1]!=0):
                                    newcord_x=xcord+1
                                    newcord_y=ycord+1
                            if (gridk[xcordnb,ycordnb-1]==0 and gridk[xcordnb,ycordnb+1]!=0):
                                    newcord_x=xcord+1
                                    newcord_y=ycord-1


                    if (xcordnb==xcord-1 and ycordnb==ycord):
                            if (gridk[xcordnb,ycordnb+1]==0 and gridk[xcordnb,ycordnb-1]==0):
                                if randint(0,1)==0:
                                    newcord_x=xcord-1
                                    newcord_y=ycord+1
                                else:
                                    newcord_x=xcord-1
                                    newcord_y=ycord-1

                            if (gridk[xcordnb,ycordnb+1]==0 and gridk[xcordnb,ycordnb-1]!=0):
                                    newcord_x=xcord-1
                                    newcord_y=ycord+1
                            if (gridk[xcordnb,ycordnb-1]==0 and gridk[xcordnb,ycordnb+1]!=0):
                                    newcord_x=xcord-1
                                    newcord_y=ycord-1
                                    
                if (pickmon!=0 and pickmon!=(len(DDk)-1)):
                    xcordnext=DDk[pickmon+1][0]
                    ycordnext=DDk[pickmon+1][1]
                    xcordprev=DDk[pickmon-1][0]
                    ycordprev=DDk[pickmon-1][1]
                    if ((xcordnext==xcord-1 and ycordnext==ycord and xcordprev==xcord and ycordprev==ycord-1) or (xcordprev==xcord-1 and ycordprev==ycord and xcordnext==xcord and ycordnext==ycord-1)): 
                        if  gridk[xcord-1,ycord-1]==0: 
                            newcord_x=xcord-1
                            newcord_y=ycord-1
                    if ((xcordnext==xcord and ycordnext==ycord-1 and xcordprev==xcord+1 and ycordprev==ycord) or (xcordprev==xcord and ycordprev==ycord-1 and xcordnext==xcord+1 and ycordnext==ycord)): 
                        if gridk[xcord+1,ycord-1]==0:   
                            newcord_x=xcord+1
                            newcord_y=ycord-1
                    if ((xcordnext==xcord+1 and ycordnext==ycord and xcordprev==xcord and ycordprev==ycord+1) or (xcordprev==xcord+1 and ycordprev==ycord and xcordnext==xcord and ycordnext==ycord+1)): 
                        if gridk[xcord+1,ycord+1]==0:
                            newcord_x=xcord+1
                            newcord_y=ycord+1
                    if ((xcordnext==xcord-1 and ycordnext==ycord and xcordprev==xcord and ycordprev==ycord+1) or (xcordprev==xcord-1 and ycordprev==ycord and xcordnext==xcord and ycordnext==ycord+1)): 
                        if gridk[xcord-1,ycord+1]==0:
                            newcord_x=xcord-1
                            newcord_y=ycord+1
                    
    for i in range(0,len(DDk)):
        
        newDDk=newDDk+[[(DDk[i][0]),(DDk[i][1]),(DDk[i][2])]]
        if (i==pickmon and (newcord_x>=0 and newcord_y>=0 and newcord_x<size and newcord_y<size)): 
            if (newcord_x!=0 or newcord_y!=0):
                newDDk[i][0]=newcord_x
                newDDk[i][1]=newcord_y
            
    for i in range(0,len(newDDk)):              #generation of the final grid using the final correct array of the protein info
        x=newDDk[i][0]  
        y=newDDk[i][1]
        c=newDDk[i][2]
        if c==2:                             # if there is a H-monomer in a specefic coordinate(x,y),set the grid point as 2 in the array
            newgridk[x,y]=2
        else :
            newgridk[x,y]=1                   # if there is a p-monomer in a specefic coordinate(x,y),set the grid point as 1 in the array
        
       
    newDDkk=newDDk
    newgridkk=newgridk                           #I change the name of the variables of the  final outcome for dont get trouble with others functions
    return newDDkk,newgridkk




"""
pivotmove montecarlo move
"""

def pivotmove(DDp,gridp):
    pick_pivot=randint(1,(len(DDp)-2))     # Randomly choose a pivot from the beads on the walk.                
                                          #The walk can be then divided in two sub-walks joined by the pivot the grid
    size=len(gridp)                        #we need to know the length of the grid for the special conditions on the edge
    newgridp=zeros((size,size))     #we create a new grid filled with zeros that we will complete at the end of the program with(or maybe no) new information 
    
    xcordpivot=DDp[pick_pivot][0]                    #x and y coordinate of the pivot monomer
    ycordpivot=DDp[pick_pivot][1]
    
    
    newDDp=[]                               #the first thing we need to do is shift the points in the grid so that the desired center of rotation moves to the origin
    newDD_subwalk=[]
        
    
    pick_side=random.random()                #Randomly choose one of the two sub-walks.                         
    pick_angle=random.random()              #Randomly choose a lattice symmetry operation(+-90º with respect to  each other)
    
    if pick_side<0.5:               #first sub-walk(we will try to ratate the monomers k+1,k+2,....N  ) 
        sub_walk=DDp[pick_pivot:(len(DDp))] 
        for i in range(pick_pivot,len(DDp)):
        
            newDD_subwalk=newDD_subwalk+[[(sub_walk[i-pick_pivot][0])-xcordpivot,(sub_walk[i-pick_pivot][1])-ycordpivot]]
            
        if pick_angle<0.5:                                   #we will rotate monomers + 90 degrees with respect to  each other
        
            theta=pi/2                                           #rotation agle is 90 degrees
            R=[[int(cos(theta)), int(-sin(theta))],[int(sin(theta)),int(cos(theta))]]        #rotation matrix
            only_rotate_subwalk=np.array(newDD_subwalk).dot(np.array(R))
            
            
        else:                #we will rotate monomers - 90 degrees with respect to  each other
                                    
            theta=-pi/2                                           #rotation agle is -90 degrees
            R=[[int(cos(theta)), int(-sin(theta))],[int(sin(theta)),int(cos(theta))]]        
            only_rotate_subwalk=np.array(newDD_subwalk).dot(np.array(R))
                                         
        for i in range(0,len(DDp)):
            if i<=pick_pivot:
                newDDp=newDDp+[[(DDp[i][0]),(DDp[i][1]),(DDp[i][2])]]
            else:
                newDDp=newDDp+[[(only_rotate_subwalk[i-pick_pivot][0])+xcordpivot,(only_rotate_subwalk[i-pick_pivot][1])+ycordpivot,(DDp[i][2])]]   
                if (((only_rotate_subwalk[i-pick_pivot][0])+xcordpivot)<0 or ((only_rotate_subwalk[i-pick_pivot][1])+ycordpivot)<0 or ((only_rotate_subwalk[i-pick_pivot][0])+xcordpivot)>=size or ((only_rotate_subwalk[i-pick_pivot][1])+ycordpivot)>=size):
                    newDDp=DDp
                    break
                if gridp[(only_rotate_subwalk[i-pick_pivot][0])+xcordpivot,(only_rotate_subwalk[i-pick_pivot][1])+ycordpivot]!=0:
                    newDDp=DDp
                    break
                         
    else:                             #second sub-walk(we will try to ratate the monomers  g-1, g-2,....0 )
        sub_walk=DDp[0:pick_pivot]
        for i in range(0,pick_pivot):
            newDD_subwalk=newDD_subwalk+[[(sub_walk[i][0])-xcordpivot,(sub_walk[i][1])-ycordpivot]]                                                  
                                         
        if pick_angle<0.5:           #we will rotate monomers + 90 degrees with respect to  each other
            if pick_pivot!=0:
                theta=pi/2                                           
                R=[[int(cos(theta)), int(-sin(theta))],[int(sin(theta)),int(cos(theta))]]          #
                only_rotate_subwalk=np.array(newDD_subwalk).dot(np.array(R))
            
        else:                #we will rotate monomers - 90 degrees with respect to  each other
            if pick_pivot!=0:
                theta=-pi/2                                           
                R=[[int(cos(theta)), int(-sin(theta))],[int(sin(theta)),int(cos(theta))]]          
                only_rotate_subwalk=np.array(newDD_subwalk).dot(np.array(R))      
        
                                         
        for i in range(0,len(DDp)):      #we generate the final correct array of the protein info(i.e we apply the translation again to the correct position for each monomer)
            if i>=pick_pivot:
                newDDp=newDDp+[[(DDp[i][0]),(DDp[i][1]),(DDp[i][2])]]
            else:
                newDDp=newDDp+[[(only_rotate_subwalk[i][0])+xcordpivot,(only_rotate_subwalk[i][1])+ycordpivot,(DDp[i][2])]]
                if (((only_rotate_subwalk[i][0])+xcordpivot)<0 or ((only_rotate_subwalk[i][1])+ycordpivot)<0 or ((only_rotate_subwalk[i][0])+xcordpivot)>=size or ((only_rotate_subwalk[i][1])+ycordpivot)>=size):
                    newDDp=DDp
                    break                             
                if grid[(only_rotate_subwalk[i][0])+xcordpivot,(only_rotate_subwalk[i][1])+ycordpivot]!=0:
                    newDDp=DDp
                    break                         
                                         
                                              
    for i in range(0,len(newDDp)):       #generation of the final grid using the final correct array of the protein info
        x=newDDp[i][0] 
        y=newDDp[i][1]
        c=newDDp[i][2]
        if c==2:                            
            newgridp[x,y]=2               # if there is a H-monomer in a specefic coordinate(x,y),set the grid point as 2 in the array
        else :
            newgridp[x,y]=1             # if there is a P-monomer in a specefic coordinate(x,y),set the grid point as 1 in the array
            
    newDDpp=newDDp                    #I change the name of the variables of the  final outcome for dont get trouble with others functions
    newgridpp=newgridp
    return newDDpp,newgridpp



    
    
                    


"""
MMC algorithim itself
"""
def MMCalgorithim(DD1,grid1):                  #we chose the variable names DD1 and grid1 for dont get any problem with other function 
    
          #we print the secuence        


                           #The following lines are the MMC algorithim
    
    size=len(grid1)
    maxIterations=22900
    KB=1                                    #We set the value of the bolztman constant in reduced units
    T=100                             #we choose 100 kelvin as testint temperature
    lengthoriginal=(len(DD1)-1)
    energyper100=[]  #We will save the energy every 100 iterations.
    energystorage=[]
    TemperatureMMC=[]
    heatcapacityMMC=[]
    for n in range(0,maxIterations+1):
            r=0.4  #random.random()   #we generate a random number between 0 and 1  #choose r=0.4 For study only the effect of kinkump    
            lengthoriginal=(len(DD1)-1) 
            energyoriginal=findenergy2(lengthoriginal,DD1)
            if r<0.5:
                trialDD,trialgrid=kinkjump(DD1,grid1)     
            else:
                trialDD,trialgrid=pivotmove(DD1,grid1)

            lengthtrial=(len(trialDD)-1)
            energytrial=findenergy2(lengthtrial,trialDD)

            if energytrial<energyoriginal:
                DD1=deepcopy(trialDD)
                grid1=deepcopy(trialgrid)
                energystorage=energystorage+[[energytrial]]
            else:

                bolztmanfactor=exp(-(energytrial-energyoriginal)/(KB*T))
                r1=random.random()
                if bolztmanfactor>r1:
                    DD1=deepcopy(trialDD)
                    grid1=deepcopy(trialgrid)
                    energystorage=energystorage+[[energytrial]]
                else:
                    energystorage=energystorage+[[energyoriginal]]
                    
            if (n % 100 == 0):
                energyper100=energyper100 + [findenergy2(len(DD1)-1,DD1)]    

            if (n%100==0 and n<=1000 and n!=0):  #temperature goes 100K to  50K  #We change the temperature every 100 iterations using n%100 
                                        #We follow our temperature scheduele so the other conditions is that n should be 
                    #less than 1000 iteraions for this first temperature loop
                TemperatureMMC=TemperatureMMC+[T]
                ener=np.nanmean((np.array(energystorage[n-90:n-1])))   #we discard 10%of trayectory
                enersquared=np.nanmean((np.array(energystorage[n-900:((n-1))]))**2)  #ener squared calculation
                heatcapacityMMC=heatcapacityMMC+[((enersquared-(ener**2))/(KB*(T**2)))] #heat capacty calculation 
                T=T-5     #we decrease the temperature


            if (n%100==0 and 1000<n<=3500): #temperature goes 50K to  25K
                TemperatureMMC=TemperatureMMC+[T]
                ener=np.nanmean((np.array(energystorage[n-90:n-1])))
                enersquared=np.nanmean((np.array(energystorage[n-90:((n-1))]))**2)
                heatcapacityMMC=heatcapacityMMC+[((enersquared-(ener**2))/(KB*(T**2)))]
                T=T-1

            if (n%100==0 and 3500<n<=5900): #temperature goes 25K to  13K
                TemperatureMMC=TemperatureMMC+[T]
                ener=np.nanmean((np.array(energystorage[n-90:n-1])))
                enersquared=np.nanmean((np.array(energystorage[n-90:((n-1))]))**2)
                heatcapacityMMC=heatcapacityMMC+[((enersquared-(ener**2))/(KB*(T**2)))]
                T=T-0.5

            if (n%100==0 and 5900<n<=8900):  #temperature goes 13K to  7K
                TemperatureMMC=TemperatureMMC+[T]
                ener=np.nanmean((np.array(energystorage[n-90:n-1])))
                enersquared=np.nanmean((np.array(energystorage[n-90:((n-1))]))**2)
                heatcapacityMMC=heatcapacityMMC+[((enersquared-(ener**2))/(KB*(T**2)))]
                T=T-0.20

            if (n%100==0 and 8900<n<=22800):  #temperature goes 13K to  0.05K
                TemperatureMMC=TemperatureMMC+[T]
                ener=np.nanmean((np.array(energystorage[n-90:n-1])))
                enersquared=np.nanmean((np.array(energystorage[n-900:((n-1))]))**2)
                heatcapacityMMC=heatcapacityMMC+[((enersquared-(ener**2))/(KB*(T**2)))]
                T=T-0.05
                
            if n==(22899):            #the last iteration,we also want to save it into the arrays
                TemperatureMMC=TemperatureMMC+[T]
                ener=np.nanmean((np.array(energystorage[n-90:n-1])))
                enersquared=np.nanmean((np.array(energystorage[n-90:((n-1))]))**2)
                heatcapacityMMC=heatcapacityMMC+[((enersquared-(ener**2))/(KB*(T**2)))]
                energyper100=energyper100 + [findenergy(len(DD)-1,DD)]
                
                  #we increase the loop count variable
            
           
            finalproteininformation=DD1
    return TemperatureMMC,heatcapacityMMC,energyper100,energystorage,finalproteininformation    #return useful variables for the report




"""
SARW protein generation:

-In the first block, several coordinates are initiated (commented individually).
-The while statement (check question 3) determines when the code can still run. This is when all conditions for a self-avoiding
 random walk are met: there should still be unoccupied space around the current position of the most recently added monomer. 
 (Ask yourself: what two options prevent the chain from going anywhere?)
     - If the above criteria are met, a random number is called, and there is a 25% chance to move in either direction
     - The code checks whether there is a free spot, and performs the step
     - the relevant functions are called for determining whether the monomer is H or P
     - The properties of the grid point are added to DD[], such that the monomer position and type are stored.
- The above continues as long as you are not at the edge. If you hit an edge, the weighting changes. The rest of the routine
  does not change, however. 
     - If we now get stuck at the edge or a corner, the code exits
     
-Plots are made (they were updated with each random walk step by plotting of a new point/H-bond)
-Energies are calculated

"""
Maxx=450                             #canvas width
Maxy=450                             #and height
ran=25
graph1 = canvas( width=Maxx, height=Maxy,title='Self Avoiding Random Walk',range=ran)
positions=points(color=color.cyan,radius=2)
M=[]                    # to plot red or white points
L=100                   # maximum length of a polymer
m=100
n=100

DD=[]                   # will contain the protein's x coordinate, y coordinate and choice of H/P monomer
size =50                 #array size x size 
size2=size*2            # net in which H P are located
nex=0    
net()                                       
k=0  
pts2=label(pos=vector(-7, -41,0), box=0) # to write No links
length=0                          # to determine length of walk
grid = zeros((size,size))         # array for particle positions
D=zeros((L,m,n))
DD=[]
storesecuence=[]
i=size//2                          # center of grid  #we use // instead of single / to have i,j as integers
j=size//2
xol=4*i-size2                     #always start at middle, x coord. transform 4*i-size2 (plotting purposes)
yol=-4*j+size2                    # y coord transform -4*j+size2
col,c=selectcol()                  #determine H/P for the first monomer
    
grid[i,j] = c                     #particle in center, grid[i,j] keeps track of the H,P-values of the grid 
M=M+[points(pos=vector(xol,yol,0),color=col, radius=5)]  #red point at center (plotting purposes)
    
    
DD=DD+[[i,j,c]]        #here we add the first point to the array that contains all protein info
if c==2:
        storesecuence=storesecuence+ ['H']
else:
        storesecuence=storesecuence+ ['P']    
    
""" In this first part, steps in the +/- x,y directions are made at random""" 

while (length<=24 and i>0 and i<size-1 and j>0 and j<size-1 and (grid[i+1,j]==0 or grid[i-1,j]==0 or grid[i,j+1]==0 or grid[i,j-1]==0)): #esa posicion i,j no haya sido utililzada,es deceir no tenga un punto rojo o blanco 

    
     
            r=random.random()             #call random no. between 0 and 1, and compare to weights to assign a direction
            if r<0.25 :                   # probability 25%
                if grid[i+1,j]==0:        # is the grid point to the right empty?
                    i+=1                  # if so: move one step to right
            elif  r>=0.25 and r<0.5:
                if grid[i-1,j]==0:
                    i-=1
            elif  r>=0.5 and r<=0.75:
                if grid[i,j+1]==0:
                    j+=1
            else:
                if grid[i,j-1]==0:
                    j-=1
             


            if grid[i,j]==0:        
                col,c=selectcol()  
                grid[i,j]=c              # add information on H,P to the grid
                length+=1                # once that grid point is occupied increase length of the protein
                DD=DD+[[i,j,c]]
                if c==2:
                            storesecuence=storesecuence+ ['H']
                else:
                            storesecuence=storesecuence+ ['P']
                            # add the new grid point, coordinates and H,p-ness to the storage array of the protein
                xp=4*i-size2             # coordinate transformation (plotting purposes)
                yp=-4*j+size2                   

                curve(pos=[vector(xol,yol,0),vector(xp,yp,0)]) #plot link last position to new one (plotting purposes)

                M=M +[points(pos=vector(xp,yp,0), color=col,radius=5)] #plot the current point (plotting purposes)
                xol=xp                          #start of new line
                yol=yp                          #y calculated is now old y (for connecting the points in the next loop iteration)
              

            while (length<=24 and j==(size-1) and i !=0 and i !=(size-1)):   # In case you are at the top row

                r1=random.random()
                if r1<0.2:                        # probability 20% to left
                    if grid[i-1,j]==0:
                            i-=1
                elif r1>0.2 and r1<0.4:           # probability 20% move right
                    if grid[i+1,j]==0:
                        i+=1
                else:                             # probability 60% move down
                     if grid[i,j-1]==0:
                        j-=1
                if grid[i,j]==0:                  #if the new point is not assigned H,P, yet...
                    col,c=selectcol()                 # choose H,P
                    grid[i,j]=c                       # grid point occupied --> assign value of H,P
                    length+=1                          #chain grew by one monomer
                    if c==2:
                            storesecuence=storesecuence+ ['H']
                    else:
                            storesecuence=storesecuence+ ['P']   
                            
                    DD=DD+[[i,j,c]]                      #store coordinates and properties of newest monomer!
                    xp=4*i-size2                         #coord transformation (plotting purposes)
                    yp=-4*j+size2

                    curve(pos=[vector(xol,yol,0),vector(xp,yp,0)])     # (plotting purposes): line connecting new point

                    M=M +[points(pos=vector(xp,yp,0), color=col,radius=5)] #(plotting purposes): add the new point 
                    xol=xp                            
                    yol=yp      # next case last row stop if corner or busy neighbors
                  
                if (i==0 or i==(size-1)) or (grid[i-1,size-1]!=0 and grid[i+1,size-1]!=0):
                    break #break out of loop, you cannot (!!) make any new monomers anymore since the endpoint is stuck.


         

            while (length<=24 and j==0 and i !=0 and i !=(size-1)):# Which row is this? this is the bottom row
                r1=random.random()  
                if r1<0.2:                        # probability 20% to left
                    if grid[i-1,j]==0:
                            i-=1
                elif r1>0.2 and r1<0.4:           # probability 20% move right
                    if grid[i+1,j]==0:
                            i+=1
                else:                             # probability 60% move up
                     if grid[i,j+1]==0:
                        j+=1

                if grid[i,j]==0:
                    col,c=selectcol()
                    grid[i,j]=c
                    length+=1
                    if c==2:
                            storesecuence=storesecuence+ ['H']
                    else:
                            storesecuence=storesecuence+ ['P']   

                    DD=DD+[[i,j,c]]
                    
                    xp=4*i-size2
                    yp=-4*j+size2

                    curve(pos=[vector(xol,yol,0),vector(xp,yp,0)])

                    M=M +[points(pos=vector(xp,yp,0), color=col,radius=5)]
                    xol=xp
                    yol=yp
                   
                if i==(size-1) or i==0 or (grid[i-1,0]!=0 and grid[i+1,0]!=0):
                    break

            while (length<=24 and i==0 and j !=0 and j !=(size-1)):
                r1=random.random()
                if r1<0.2:
                    if grid[i,j-1]==0:
                        j-=1
                elif r1>0.2 and r1<0.4:
                    if grid[i,j+1]==0:
                        j+=1
                else:
                    if grid[i+1,j]==0:
                        i+=1

                if grid[i,j]==0:
                    col,c=selectcol()
                    grid[i,j]=c
                    length+=1
                    if c==2:
                            storesecuence=storesecuence+ ['H']
                    else:
                            storesecuence=storesecuence+ ['P']  

                    DD=DD+[[i,j,c]]
                    
                    xp=4*i-size2
                    yp=-4*j+size2

                    curve(pos=[vector(xol,yol,0),vector(xp,yp,0)])


                    M=M +[points(pos=vector(xp,yp,0), color=col,radius=5)]
                    xol=xp
                    yol=yp
                    
                if j==(size-1) or j==0 or (grid[0,j+1]!=0 and grid[0,j-1]!=0):
                    break

            while (length<=24 and i==(size-1) and j !=0 and j !=(size-1)): 
                r1=random.random()

                if r1<0.2:
                    if grid[i,j-1]==0:
                        j-=1
                elif r1>0.2 and r1<0.4:
                    if grid[i,j+1]==0:
                        j+=1
                else:
                    if grid[i-1,j]==0:
                        i-=1

                if grid[i,j]==0:
                    col,c=selectcol()
                    grid[i,j]=c
                    length+=1
                    col,c=selectcol()
                    if c==2:
                            storesecuence=storesecuence+ ['H']
                    else:
                            storesecuence=storesecuence+ ['P']
                    DD=DD+[[i,j,c]]
                    
                    xp=4*i-size2
                    yp=-4*j+size2

                    curve(pos=[vector(xol,yol,0),vector(xp,yp,0)])

                    M=M +[points(pos=vector(xp,yp,0), color=col,radius=5)]
                    xol=xp
                    yol=yp
                   
                if j==(size-1) or (grid[size-1,j+1]!=0 and grid[size-1,j-1]!=0):
                    break    
                    
                    
findenergy(len(DD)-1,DD)            #callinf findenergy function we obtain the plot with the yellow points(energy link represenation)
print(storesecuence)

#note:sometimes when i run it the plot does not appear,for fix that i just click in
#kenel and i clean and reestar all outpots,then i run it again.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'P', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'P', 'H', 'P', 'H', 'H']


In [8]:
temperaturearrayforplot,heatcapacityarrray,energyevery100iterations,energia,fpconfing=MMCalgorithim(DD,grid)     #run it to start the MMC and get the final variable of interest
#running this line will also print the energy per 100 iterations
print(energyevery100iterations)


[-2, -1, 0, -2, -3, -1, -1, -2, -3, -2, -1, -2, -1, -2, -3, -3, -1, -2, -1, -2, 0, -2, -4, -2, -3, -3, -3, -1, -4, -5, 0, -1, -2, -3, -3, -2, -3, -3, -2, -2, -6, -2, -3, -2, -1, -2, -1, -2, -1, -2, -2, -1, -2, -2, -3, -2, -3, -3, -2, -4, -2, -4, -2, -2, -2, -5, -6, -3, -3, -4, -5, -5, -7, -5, -2, -3, -4, -6, -6, -3, -6, -4, -6, -5, -5, -6, -5, -7, -6, -5, -6, -6, -5, -7, -8, -6, -5, -3, -6, -5, -8, -6, -7, -4, -8, -8, -8, -3, -6, -7, -5, -3, -6, -5, -3, -4, -5, -7, -3, -5, -5, -2, -3, -3, -4, -2, -1, -1, -3, -3, -3, -2, -3, -3, -4, -5, -5, -6, -2, -4, -5, -4, -3, -4, -4, -5, -4, -3, -7, -5, -6, -5, -5, -4, -4, -3, -4, -1, -3, -3, -4, -3, -1, -2, -3, -2, -4, -3, -4, -4, -4, -5, -4, -6, -9, -4, -3, -2, -4, -4, -5, -4, -2, -3, -2, -3, -4, -6, -4, -3, -2, -3, -3, -3, -1, -3, -2, -2, -5, -6, -3, -5, -5, -5, -5, -6, -6, -8, -6, -4, -7, -7, -5, -5, -4, -5, -4, -3, -5, -5, -5, -5, -6, -7, -7, -7, -7, -7, -7, -2, -7]


In [9]:
#final configuration of the protein
plotfinalconformation(fpconfing)    
findenergy(len(fpconfing)-1,fpconfing)

<IPython.core.display.Javascript object>

-7

In [ ]:
#normal plot
import matplotlib.pyplot as plt
import numpy as np

plt.plot(temperaturearrayforplot, heatcapacityarrray)

plt.xlim(0.8,7)
#plt.xlim(5, 0)   # decreasing time
plt.ylim(-0.2,16)

#plt.xlim(5, 0)   # decreasing time

plt.xlabel('Temperature(K)')
plt.ylabel('Heat Capacity(J/K))')
plt.title('Heat Capacity vs Temperature ')
#plt.grid(True)
plt.grid(False)

plt.show()

In [ ]:
plt.scatter(temperaturearrayforplot,heatcapacityarrray)


plt.xlim(0.8, 7)
#plt.xlim(5, 0)   # decreasing time
plt.ylim(-1,16)

plt.xlabel('Temperature(K)')
plt.ylabel('Heat Capacity(J/K))')
plt.title('Scatter plot Heat Capacity vs Temperature ')
#plt.grid(True)
plt.grid(False)

plt.show()